# Homologous cell-types definition
## To match macrophage cell-types between adult mice and zebrafish

In [ ]:
#%matplotlib nbagg
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import pandas as pd
from os import listdir
from os.path import isfile, join
import re
import anndata
import seaborn as sns

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()

from matplotlib.colors import LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list(name='gene_cmap', colors=['lightgrey', 'thistle', 'red', 'darkred']) 

sc.settings.set_figure_params(dpi=80, color_map='viridis', vector_friendly=False,  dpi_save=300)
from matplotlib import font_manager
font_manager.fontManager.addfont('/cluster/groups/Wangxiaonan/share/yingjun/arial.ttf')
plt.rcParams['font.sans-serif'] = 'Arial'

# Scoring using top 100 DEG

## preprocess

In [ ]:
adata = sc.read('./Final_write/mouse_mac_final.h5ad')

In [ ]:
adata = anndata.AnnData(X=adata.raw.X.A,var=adata.raw.var,obs=adata.obs,obsm=adata.obsm)
# adata.write('./write/Macro_outerRaw.h5ad')

In [ ]:
# Scale
sc.pp.scale(adata,max_value=10)

In [ ]:
np.max(adata.X)

## Start scoring

In [ ]:
gdict = pd.read_csv('./Final_Files/TransedMouse100Marker_Outer',index_col=0)

In [ ]:
gdict = gdict.to_dict()

In [ ]:
for k, v in gdict.items():
    sc.tl.score_genes(adata, gene_list=v.values(), score_name='Score_'+k, use_raw=False)

In [ ]:
sname = [x for x in adata.obs_keys() if 'Score_' in x]

In [ ]:
adata.obs['t2'] = np.std(adata.obs[sname], axis=1)

In [ ]:
s2 = adata.obs[sname].max(axis=1)

In [ ]:
adata.obs['s2'] = s2

In [ ]:
sc.pl.umap(adata,color='s2',cmap='bwr',title='Score',save='_s2.pdf')

In [ ]:
adata.obs['Score']=adata.obs.s2

In [ ]:
ax = pd.DataFrame(adata.obs['Score']).plot.kde(xlim=(-0.5,1.5))
ax.axvline(0.05, linestyle='--', color='orange')
plt.savefig('s2_density.pdf',bbox_inches='tight')

In [ ]:
Tn = adata.obs[sname].idxmax(axis=1)

In [ ]:
adata.obs['Tn'] = [re.sub('Score_', '',x) for x in Tn]

In [ ]:
tk = [adata.obs.Tn[x] if adata.obs.s2[x]>0.05 else 'NA' for x in adata.obs_names]

In [ ]:
adata.obs['Tn2'] = tk

In [ ]:
adata.obs['Tn'].value_counts()

In [ ]:
adata.obs['Tn2'].value_counts()

In [ ]:
adata2 = adata[adata.obs.Tn2!='NA']

In [ ]:
cat = ['Ma-TR-Cd163', 'Ma-TR-Cd83+', 'Ma-TR-Cd83-', 'Ma-MD-Mpeg1',
       'Ma-MD-Prdx1', 'Ma-MD-Cxcl3', 'Ma-MD-Isg15', 'Ma-MD-Ccl2', 'Ma-Fn1', 'Ma-Fibro',
       'Ma-Endo', 'Ma-Cycling', 'Ma-lowGC']
adata2.obs['Tn2'] = adata2.obs.Tn2.astype('category').cat.reorder_categories(cat)

In [ ]:
sc.pl.umap(adata2,color=['Tn2'],wspace=0.5,title='Adult_Mice')

In [ ]:
col = ['#d6bcc0', '#bb7784', '#8e063b','#00008B', '#4a6fe3', '#8595e1','#FF8C00','#634040', '#ADFF2F', '#32CD32', '#006400', '#C0C0C0', '#bec1d4']
adata2.uns['Tn2_colors'] = col

In [ ]:
sc.pl.umap(adata2,color=['Tn2'],size=3,title='Zebrafish CellType')          

In [ ]:
sc.tl.rank_genes_groups(adata2, 'Tn2', n_genes=adata2.shape[1], key_added='DE_Tn2')

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata2,key='DE_Tn2',groupby='Tn2', cmap=cmap, swap_axes=True, standard_scale='var',dendrogram=False,n_genes=3,title='Zebrafish',
                                save="fish_markers.pdf"
                               )

In [ ]:
adata2.write('./write/Macro_outerRaw_CT3scored_filtered.h5ad')

In [ ]:
adata = sc.read('./write/Macro_outerRaw_scored_filtered.h5ad')